# Turbine Models Library Pre-Processing Tools

The [turbine-models package](https://github.com/NREL/turbine-models/tree/main) hosts wind turbine data for a variety of wind turbines and has tools that can streamline the process to run new turbines with the [PySAM Windpower model](https://nrel-pysam.readthedocs.io/en/main/modules/Windpower.html) or [FLORIS](https://github.com/NREL/floris/tree/main).

The full list of turbine models available in the turbine-models library can be found [here](https://github.com/NREL/turbine-models/blob/main/turbine_models/supported_turbines.py)

H2Integrate has preprocessing tools that leverage the functionality available in the turbine-models library. The function `export_turbine_to_pysam_format()` will save turbine model specifications formatted for the PySAM Windpower model. The PySAM Windpower model is wrapped in H2I and can be utilized with the "pysam_wind_plant_performance" model. Example usage of the `export_turbine_to_pysam_format()` function is demonstrated in the following section using Example 8.


## Turbine Model Pre-Processing with PySAM Windpower Model
Example 8 (`08_wind_electrolyzer`) currently uses an 8.3 MW turbine. In the following sections we will demonstrate how to:

1. Save turbine model specifications for the NREL 5 MW turbine in the PySAM Windpower format using `export_turbine_to_pysam_format()`
2. Load the turbine model specifications for the NREL 5 MW turbine and update performance parameters for the wind technology in the `tech_config` dictionary for the NREL 5 MW turbine.
3. Run H2I with the updated tech_config dictionary, showcasing two different methods to run H2I with the NREL 5 MW turbine:
   - initializing H2I with a dictionary input
   - saving the updated tech_config dictionary to a new file and initializing H2I by specifying the filepath to the top-level config file.


We'll start off by importing the required modules and packages:

In [8]:
import os
import numpy as np


from h2integrate import EXAMPLE_DIR
from h2integrate.core.utilities import load_yaml
from h2integrate.core.inputs.validation import load_tech_yaml
from h2integrate.preprocess.wind_turbine_file_tools import export_turbine_to_pysam_format
from h2integrate.core.h2integrate_model import H2IntegrateModel

Load the tech config file that we want to update the turbine model for:

In [9]:
# Load the tech config file
tech_config_path = EXAMPLE_DIR / "08_wind_electrolyzer" / "tech_config.yaml"
tech_config = load_tech_yaml(tech_config_path)

This example uses the "pysam_wind_plant_performance" performance model for the wind plant. Currently, the performance model is using an 8.3MW wind turbine with a rotor diameter of 196 meters and a hub-height of 130 meters. This information is defined in the `tech_config` file:

```yaml
technologies:
  wind:
    performance_model:
      model: "pysam_wind_plant_performance"
    cost_model:
      model: "atb_wind_cost"
    model_inputs:
      performance_parameters:
        num_turbines: 100
        turbine_rating_kw: 8300
        rotor_diameter: 196.
        hub_height: 130.
        create_model_from: "default"
        config_name: "WindPowerSingleOwner"
        pysam_options: !include "pysam_options_8.3MW.yaml"
        run_recalculate_power_curve: False
        layout:
          layout_mode: "basicgrid"
          layout_options:
            row_D_spacing: 10.0
            turbine_D_spacing: 10.0
            rotation_angle_deg: 0.0
            row_phase_offset: 0.0
            layout_shape: "square"
      cost_parameters:
        capex_per_kW: 1500.0
        opex_per_kW_per_year: 45
        cost_year: 2019
```


If we want to replace the 8.3 MW turbine with the NREL 5 MW turbine, we can do so using the `export_turbine_to_pysam_format()` function:

In [10]:
turbine_name = "NREL_5MW"

turbine_model_fpath = export_turbine_to_pysam_format(turbine_name)

print(turbine_model_fpath)

/Users/gstarke/Documents/Research_Programs/H2I/H2Integrate/library/pysam_options_NREL_5MW.yaml


In [11]:
# Load the turbine model file formatted for the PySAM Windpower module
pysam_options = load_yaml(turbine_model_fpath)
pysam_options

{'Turbine': {'wind_turbine_max_cp': 0.481305875,
  'wind_turbine_ct_curve': [0.0,
   0.0,
   0.0,
   1.132034888,
   0.999470963,
   0.917697381,
   0.860849503,
   0.815371198,
   0.811614904,
   0.807939328,
   0.80443352,
   0.800993851,
   0.79768116,
   0.794529244,
   0.791495834,
   0.788560434,
   0.787217182,
   0.787127977,
   0.785839257,
   0.783812219,
   0.783568108,
   0.783328285,
   0.781194418,
   0.777292539,
   0.773464375,
   0.769690236,
   0.766001924,
   0.762348072,
   0.758760824,
   0.755242872,
   0.751792927,
   0.748434131,
   0.745113997,
   0.717806682,
   0.672204789,
   0.63831272,
   0.610176496,
   0.585456847,
   0.563222111,
   0.542912273,
   0.399312061,
   0.310517829,
   0.248633226,
   0.203543725,
   0.169616419,
   0.143478955,
   0.122938861,
   0.106515296,
   0.093026095,
   0.081648606,
   0.072197368,
   0.064388275,
   0.057782745,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
   0.0,
 

In [12]:
# Create dictionary of updated inputs for the new turbine formatted for
# the "pysam_wind_plant_performance" model
updated_parameters = {
    "turbine_rating_kw": np.max(pysam_options["Turbine"].get("wind_turbine_powercurve_powerout")),
    "rotor_diameter": pysam_options["Turbine"].pop("wind_turbine_rotor_diameter"),
    "hub_height": pysam_options["Turbine"].pop("wind_turbine_hub_ht"),
    "pysam_options": pysam_options,
}

# Update wind performance parameters with model from PySAM
tech_config["technologies"]["wind"]["model_inputs"]["performance_parameters"].update(
    updated_parameters
)

# The technology input for the updated wind turbine model
tech_config["technologies"]["wind"]["model_inputs"]["performance_parameters"]

{'num_turbines': 100,
 'turbine_rating_kw': np.float64(5000.0),
 'rotor_diameter': 126,
 'hub_height': 90,
 'create_model_from': 'default',
 'config_name': 'WindPowerSingleOwner',
 'pysam_options': {'Turbine': {'wind_turbine_max_cp': 0.481305875,
   'wind_turbine_ct_curve': [0.0,
    0.0,
    0.0,
    1.132034888,
    0.999470963,
    0.917697381,
    0.860849503,
    0.815371198,
    0.811614904,
    0.807939328,
    0.80443352,
    0.800993851,
    0.79768116,
    0.794529244,
    0.791495834,
    0.788560434,
    0.787217182,
    0.787127977,
    0.785839257,
    0.783812219,
    0.783568108,
    0.783328285,
    0.781194418,
    0.777292539,
    0.773464375,
    0.769690236,
    0.766001924,
    0.762348072,
    0.758760824,
    0.755242872,
    0.751792927,
    0.748434131,
    0.745113997,
    0.717806682,
    0.672204789,
    0.63831272,
    0.610176496,
    0.585456847,
    0.563222111,
    0.542912273,
    0.399312061,
    0.310517829,
    0.248633226,
    0.203543725,
    0.1

### Option 1: Run H2I with dictionary input

In [13]:
# Create the top-level config input dictionary for H2I
h2i_config = {
    # "name": "H2Integrate Config",
    # "system_summary": f"Updated hybrid plant using {turbine_name} turbine",
    "driver_config": EXAMPLE_DIR / "08_wind_electrolyzer" / "driver_config.yaml",
    "technology_config": tech_config,
    "plant_config": EXAMPLE_DIR / "08_wind_electrolyzer" / "plant_config.yaml",
}

# Create a H2Integrate model with the updated tech config
h2i = H2IntegrateModel(h2i_config)

# Run the model
h2i.run()

# Get LCOE of wind plant
wind_lcoe = h2i.model.get_val("finance_subgroup_electricity_profast.LCOE", units="USD/MW/h")
print(f"Wind LCOE is ${wind_lcoe[0]:.2f}/MWh")

# Get LCOH of wind/electrolyzer plant
lcoh = h2i.model.get_val("finance_subgroup_hydrogen.LCOH_produced_profast_model", units="USD/kg")
print(f"LCOH is ${lcoh[0]:.2f}/kg")

Wind LCOE is $68.18/MWh
LCOH is $6.84/kg


### Option 2: Save new tech_config to file and run H2I from file

In [14]:
from h2integrate.core.utilities import write_readable_yaml

# Define a new filepath for the updated tech config
tech_config_path_new = EXAMPLE_DIR / "08_wind_electrolyzer" / f"tech_config_{turbine_name}.yaml"

# Save the updated tech config to the new filepath
write_readable_yaml(tech_config, tech_config_path_new)

# Load in the top-level H2I config file
h2i_config_path = EXAMPLE_DIR / "08_wind_electrolyzer" / "wind_plant_electrolyzer.yaml"
h2i_config_dict = load_yaml(h2i_config_path)

# Define a new filepath for the updated top-level config
h2i_config_path_new = (
    EXAMPLE_DIR / "08_wind_electrolyzer" / f"wind_plant_electrolyzer_{turbine_name}.yaml"
)

# Update the technology config filepath in the top-level config with the updated
# tech config filename
h2i_config_dict["technology_config"] = tech_config_path_new.name

# Save the updated top-level H2I config to the new filepath
write_readable_yaml(h2i_config_dict, h2i_config_path_new)

# Change the CWD to the example folder since filepaths in h2i_config_dict are relative
# to the "08_wind_electrolyzer" folder
os.chdir(EXAMPLE_DIR / "08_wind_electrolyzer")

# Create a H2Integrate model with the updated tech config
h2i = H2IntegrateModel(h2i_config_path_new.name)

# Run the model
h2i.run()

# Get LCOE of wind plant
wind_lcoe = h2i.model.get_val("finance_subgroup_electricity_profast.LCOE", units="USD/MW/h")
print(f"Wind LCOE is ${wind_lcoe[0]:.2f}/MWh")

# Get LCOH of wind/electrolyzer plant
lcoh = h2i.model.get_val("finance_subgroup_hydrogen.LCOH_produced_profast_model", units="USD/kg")
print(f"LCOH is ${lcoh[0]:.2f}/kg")

Wind LCOE is $68.18/MWh
LCOH is $6.84/kg


## Turbine Model Pre-Processing with FLORIS

The function `export_turbine_to_floris_format()` will save turbine model specifications formatted for FLORIS. 